### Sesión de Spark

En Spark es necesario crear una sesión antes de empezar a trabajar con los dataframes, y también se crea un contexto.

Usualmente se usan las variables **spark** y **sc**

In [0]:
%python
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
sc=spark.sparkContext.getOrCreate()

In [0]:
spark

In [0]:
sc

Vamos a crear un diccionario con algunos datos de ejemplo.

In [0]:
%python
datos = {
    "Name": ["John", "Sally", "Alyssa"],
    "Salary": [50000, 120000, 80000],
    "Age": [34, 45, 27]
}

### Paralelización

Para convertir una estructura de datos o una colección en un dataframe de **Spark**, hay que distribuirlo, lo cual se realiza con la instrucción **paralellize**, primero nos crea un RDD o **Dataset Distribuido Resiliente**.


In [0]:
mi_rdd=sc.parallelize(list(zip(datos["Name"], datos["Salary"], datos["Age"])))

In [0]:
mi_rdd

### Creación del Dataframe en Spark

Luego con la sesión de spark usamos **createDataFrame** y ahora si, tenemos un **Dataframe** de Spark.

En particular, mira el output de la celda e identifica el **tipo**.

In [0]:
columns = ["Name", "Salary", "Age"]
df_spark=spark.createDataFrame(mi_rdd,schema=columns)

In [0]:
df_spark.show()

### Trabajando con Pyspark, Agregar Columnas

Primero vamos a crear nuevas columnas, limpiar las que no necesitamos con **drop**

Podemos agregar columnas constantes con **lit** o hacer operaciones entre columnas.

In [0]:
from pyspark.sql.functions import lit
df=df_spark.drop("Salary")

In [0]:
df=df.withColumn("valorliteral", lit(2))

calculando el producto entre dos columnas

In [0]:
from pyspark.sql.functions import col
df=df_spark.withColumn("productocolumnas", col("age")*col("salary"))

In [0]:
df.show()

In [0]:
df.select(col("Name")).show()

In [0]:
from pyspark.sql.functions import desc,asc
df.select('*').orderBy(asc("Age")).show()

In [0]:
from pyspark.sql.functions import sum
df=df.withColumn("SalaryInteger",col("Salary").cast("Integer"))
df.agg(sum(col("Salary")).alias("Suma_Salario")).show()

## Lectura de Archivos

Spark tiene la funcionalidad para leer archivos parquet, csv, orc, delta, entre otros.

Se usa la función **spark.read.formato**

In [0]:
csv_content = """name,gender,age,weight,job
Felipe,M,15,80,teacher
Maria,F,18,45,student
Juliana,F,23,50,nurse
Andres,M,22,87,engineer
Ivan,M,54,90,doctor
"""

file_path = "dbfs:/FileStore/tables/Archivo_Ejemplo.csv"

dbutils.fs.put(file_path, csv_content, overwrite=True)

In [0]:
%fs
head /FileStore/tables/Archivo_Ejemplo.csv

In [0]:
%python
df_tabla=spark.read.csv("/FileStore/tables/Archivo_Ejemplo.csv",header=True,inferSchema=True)

In [0]:
df_tabla.show()

In [0]:
df_tabla.explain()

In [0]:
display(df_tabla)

### Accediendo a Propiedades del Dataframe

Podemos acceder a datos estadísticos del dataframe utilizando **describe** y tambien **printSchema**

In [0]:
df_tabla.describe().show()

In [0]:
df_tabla.printSchema()

In [0]:
df_tabla.explain()

### Creando Tablas Temporales para Trabajar con SQL

Se usa el método **createOrReplaceTempView** que nos va a permitir hacer queries.

In [0]:
%python
df_tabla.createOrReplaceTempView("Tabla_Hive")

In [0]:
%sql
SELECT * FROM Tabla_Hive


In [0]:
%sql
Select count(*) as ocurrencia, gender as genero from Tabla_Hive GROUP BY gender

### Guardando Tablas en el Catálogo

Vamos a crear una base de datos y posteriormente a crear una tabla.

In [0]:
%sql
DROP SCHEMA IF EXISTS Salarios CASCADE;
CREATE DATABASE IF NOT EXISTS Salarios

In [0]:
df_tabla.write.mode("overwrite").saveAsTable("Salarios.tablasalarios")


Lo mismo pero con SQL

In [0]:
%sql
CREATE TABLE IF NOT EXISTS salarios.tablasalarios3 AS SELECT * FROM Tabla_Hive

In [0]:
%sql
SELECT * FROM salarios.tablasalarios3

### Transición de Spark a Pandas y Viceversa

Podemos transmutar de pandas a spark y viceversa, con los métodos **.toPandas()**, o **to_pandas_on_spark()** o **to_koalas()**


In [0]:
df_pandas=df_tabla.toPandas()

In [0]:
df_pandas

Hay una versión paralelizada de **pandas** denominada **koalas**, tiene los beneficios de Pandas, con la paralelización y optimizaciones de Spark.

In [0]:
df_koalas=df_tabla.to_koalas()

In [0]:
df_koalas

De regreso de **Pandas** a **Spark**.

In [0]:
spark_df_from_pandas=spark.createDataFrame(df_pandas)

### Funciones y Mapeo

Puedes aplicar una función a una columna o a todo el dataframe en Spark usando **UDFs** o **User defined functions**, o usando funciones predefinidas usando **map o flatmap**



In [0]:
import datetime
from pyspark.sql.types import IntegerType

current_year = datetime.datetime.now().year

def calculate_year_of_birth(age):
    return current_year - int(age)

year_of_birth_udf = udf(calculate_year_of_birth, IntegerType())

In [0]:
df_con_udf = spark_df_from_pandas.withColumn("yearofbirth", year_of_birth_udf(col("Age")))

In [0]:
df_con_udf.show()